In [ ]:
import geopandas as gpd
import json
import math
import numpy as np
import os
import pandas as pd
import plotly.express as px


# PARAMETERS
dataFolderName = 'data'
geoJsonFolder = dataFolderName+'/geoJson/'

In [ ]:
# Create geo-data, year by year, correcting input data to make them compatible with our data

map_df = {} # dictionary, year as key
map_df[2014] = gpd.read_file(f'{dataFolderName}/province_shapes/Prov01012014_g/Prov01012014_g_WGS84.shp')
map_df[2014]['DEN_PCM'] = map_df[2014]['DEN_PROV']  # duplicate this column to make the dataframe compliant with those of subsequent years 
map_df[2014].loc[ map_df[2014].DEN_PCM=="Forlì-Cesena","DEN_PCM" ] = "Forli'-Cesena"

for year in range(2015,2018):
    fp = f'{dataFolderName}/province_shapes/ProvCM01012017_g/ProvCM01012017_g_WGS84.shp' # data updated to 1st Jan 2017 work for our purposes
    map_df[year] = gpd.read_file(fp) #reading the file stored in variable fp
    map_df[year].loc[ map_df[year].DEN_PCM=="Aosta","DEN_PCM" ] = "Valle d'Aosta/Vallée d'Aoste"
    map_df[year].loc[ map_df[year].DEN_PCM=="Massa Carrara","DEN_PCM" ] = "Massa-Carrara"
    map_df[year].loc[ map_df[year].DEN_PCM=="Bolzano","DEN_PCM" ] = "Bolzano/Bozen"



# Change the coordinate system

print('Input Coordinate Reference System: \t ' + str(map_df[2014].crs)) # print the Coordinate Reference System (CRS), EPSG:32632 is WGS 84 / UTM zone 32N (WGS=World Geodetic System, UTM=Universal Transverse Mercator)

# Function to convert (project) coordinates to latitude/longitude
def convertCrsToLatLong(inputGeopandasDf):
    outputGeopandasDf = inputGeopandasDf.set_geometry("geometry") # The original geometry column is replaced with "geometry" (if it was different).
    outputGeopandasDf = outputGeopandasDf.to_crs("EPSG:4326")
    return outputGeopandasDf

# Create a GeoJson file from the *.shp and read it
geoJsonData = {}
if not os.path.exists(geoJsonFolder):
    os.makedirs(geoJsonFolder)
for year in range(2014, 2018):
    map_df[year] = convertCrsToLatLong(map_df[year])
    geoJsonPathThisYear = geoJsonFolder+str(year)+'.json'
    map_df[year].to_file(geoJsonPathThisYear, driver="GeoJSON")
    with open(geoJsonPathThisYear, encoding="utf-8") as geofile:
        geoJsonData[year] = json.load(geofile)

print('Converted Coordinate Reference System: \t ' + str(map_df[2014].crs) + '\n') # print the Coordinate Reference System (CRS)

# Plot the map (unnecessary, just to show that it works)
map_df[2014].plot()

print(map_df[2014].head())  # NOTE: 'DEN_PCM' COLUMNS contains distinct values
for year in map_df:
    print(f'Year {year}:\t' + str(len(map_df[year])) + ' ' + str(len(map_df[year]['DEN_PCM'].drop_duplicates())))
    # # Print duplicates:
    # print(map_df[year] [map_df[year].duplicated('DEN_PCM')])

In [ ]:
# Load data from CSV to the program
dataFileName = dataFolderName + '/DCSC_RACLI_01092021113430630.csv'
df = pd.read_csv(dataFileName).drop_duplicates()

# Transform the data and remove useless columns
df['Territorio'] = df['Territorio'].str.replace(' / ','/')
df = df.drop('TIPO_DATO7', axis=1) # always the same (HOUWAG_ENTEMP_AV_MI)
df = df.drop('Tipo dato', axis=1)  # always the same (Retribuzione lorda oraria per ora retribuita delle posizioni lavorative dipendenti in euro (media).)
df = df.drop(['SEXISTAT1', 'ETA1_A','PROFILO_PROF','CLLVT','Seleziona periodo'], axis=1)  # ridondance of information
df = df[df['Flag Codes'] != 'c'].drop(['Flags','Flag Codes'], axis=1) # delete incomplete data and drop columns with corresponding flag ('c' is the flag for hidden data)

# Get data about territory (discard the sector)
df_territory = df.query('`Ateco 2007`=="TOTALE"').drop(['Ateco 2007', 'ATECO_2007'], axis=1)

# Consider only data about provinces
df_province = df_territory[df_territory['ITTER107'].str.contains('.{5}')].drop('ITTER107', axis=1)   # for provinces, 'ITTER107' code is 5 chars long
df_province.loc[df_province['Territorio']=="Forlì-Cesena", "Territorio"] = "Forli'-Cesena"           # transform data for consistency with datasets of geocoords

print(df_province["Territorio"].drop_duplicates().sort_values().tolist())
print(df_province.head())

# Create a dictionary having years as keys
df_province_years = {year: df_province.query(f'TIME=={year}').drop('TIME', axis=1).drop_duplicates() for year in range(2014,2018)}
print(pd.DataFrame(df_province_years[2014]).head())

In [ ]:
# Test data for the map
test_year = 2017        # year to use as test
df_test = df_province_years[test_year].query("Sesso=='totale' & `Classe di età`=='totale' & `Qualifica contrattuale`=='totale' & `Classe di dipendenti`=='totale'")    \
                                      .drop(['Sesso', 'Classe di età', 'Qualifica contrattuale', 'Classe di dipendenti'], axis=1)
geoJsonData_test = geoJsonData[test_year]

print(df_province_years[test_year]["Territorio"].drop_duplicates().sort_values().tolist())
print()
print(map_df[test_year]["DEN_PCM"].drop_duplicates().sort_values().tolist())
print()
print('\n\t\tproperties.DEN_PCM:\n' + str(sorted([prov['properties']['DEN_PCM'] for prov in geoJsonData_test['features']])) + '\n\n')

print(df_test.head())
print(geoJsonData_test.keys())
print('\tfeatures:\t' + str(geoJsonData_test['features'][0].keys()))
print('\t\tproperties:\t' + str(geoJsonData_test['features'][0]['properties'].keys()))

print(geoJsonData_test['features'][0]['properties'].values())
# print(geoJsonData_test['features'][0]['geometry']['coordinates'])
print(geoJsonData_test['features'][1]['properties'].values())


print('Same lenght of dataframes? ' + str(len(df_test)==len(geoJsonData_test['features'])) + ' (' + str(len(df_test)) + ')')

print(geoJsonData_test['features'][0]['properties'].values())

In [ ]:
# Prepare test data for the map

avgSalaryInItaly = round(100*df_territory.query(f"Territorio=='Italia' & Sesso=='totale' & `Classe di età`=='totale' & `Qualifica contrattuale`=='totale' & `Classe di dipendenti`=='totale'")['Value'].mean())/100  # round(100*..)/100 is used to have two decimal digits
print(f"Average salary in Italy over years: " + str(avgSalaryInItaly) + " €/hr")

# Group data
valueCountedData = np.floor(df_test["Value"]).astype(int).value_counts().sort_index(ascending=False)
groupedData = { floored: {territory: value for territory, value in df_test[['Territorio', 'Value']].to_numpy() if math.floor(value)==floored}    \
                    for floored in np.floor(df_test["Value"]).astype(int).drop_duplicates() }
print("Floor\tCount\n" + str(valueCountedData))
print(str(groupedData).replace('},',']\n'))

# Categorization in categories
oldCategory=0
salaryCategoryBorders = range(9,20,2) # 5 categories
for category in salaryCategoryBorders:
    numberProvinceInThisCategory = sum([valueCountedData[key] for key in np.intersect1d(valueCountedData.keys().tolist(), range(oldCategory,category))])
    df_test.loc[(oldCategory<=df_test['Value']) & (df_test['Value']<category), "SalaryCategory"] =  \
        (f"{oldCategory} ≤ " if oldCategory > salaryCategoryBorders[0] else "        ")             \
        + ".."                                                                                      \
        + (f" < {category}"  if category < salaryCategoryBorders[-1] else "        ")               \
        + f"  €/hr\t({numberProvinceInThisCategory} provinces)"
    oldCategory = category
del oldCategory, numberProvinceInThisCategory


# sort (needed to respect the color scale if categorization is used)
df_test.sort_values(by=['Value'], ascending=False, inplace=True)

In [ ]:
# Choropleth by categories

fig = px.choropleth(
    data_frame=df_test, 
    geojson=geoJsonData_test, 
    locations='Territorio',              # name of dataframe column
    featureidkey='properties.DEN_PCM',   # path to field in GeoJSON feature object with which to match the values passed in to locations
    color='SalaryCategory',
    # color_continuous_scale="rdylgn",                                                                          # for continuos scale of colors
    color_discrete_sequence=['#4dac26', '#b8e186', '#fefee9', '#f1b6da', '#d01c8b'],      # for discrete scale of colors
    center={"lat": 42, "lon": 13},
    projection='mercator',
    labels={'SalaryCategory': 'Average hourly gross salary'},
    hover_name='Territorio',
    hover_data={'Value':True, 'SalaryCategory':False, 'Territorio': False}          # TODO: improve this (see "hovertemplate")
)
fig.update_traces(marker=dict(opacity=1, line=dict(color='black', width=0.1)))      # TODO: look for "hovertemplate, https://plotly.com/python/reference/choropleth/#choropleth-hovertemplate"
fig.update_layout(plot_bgcolor='white', font=dict(color='dimgray'), title='Salaries in private companies')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_geos(showcountries=False, showcoastlines=False, showland=False, fitbounds="locations")
fig.show("notebook")

In [ ]:
# Choropleth mapbox
fig = px.choropleth_mapbox(
    df_test,
    geojson=geoJsonData_test,       
    color='Value',                      # name of a dataframe column
    locations='Territorio',             # name of a dataframe column
    featureidkey="properties.DEN_PCM",
    center={"lat": 42, "lon": 13},
    mapbox_style="carto-positron",
    zoom=4.4
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_geos(showcountries=False, showcoastlines=False, showland=False, fitbounds="locations")
fig.show()

In [ ]:
# NOTE : CODE DUPLICATION!

# Create geo-data, year by year, correcting input data to make them compatible with our data
map_df = {} # dictionary, year as key
map_df[2014] = gpd.read_file(f'{dataFolderName}/province_shapes/Prov01012014_g/Prov01012014_g_WGS84.shp')
map_df[2014]['DEN_PCM'] = map_df[2014]['DEN_PROV']  # duplicate this column to make the dataframe compliant with those of subsequent years 
map_df[2014].loc[ map_df[2014].DEN_PCM=="Forlì-Cesena","DEN_PCM" ] = "Forli'-Cesena"

for year in range(2015,2018):
    fp = f'{dataFolderName}/province_shapes/ProvCM01012017_g/ProvCM01012017_g_WGS84.shp' # data updated to 1st Jan 2017 work for our purposes
    map_df[year] = gpd.read_file(fp) #reading the file stored in variable fp
    map_df[year].loc[ map_df[year].DEN_PCM=="Aosta","DEN_PCM" ] = "Valle d'Aosta/Vallée d'Aoste"
    map_df[year].loc[ map_df[year].DEN_PCM=="Massa Carrara","DEN_PCM" ] = "Massa-Carrara"
    map_df[year].loc[ map_df[year].DEN_PCM=="Bolzano","DEN_PCM" ] = "Bolzano/Bozen"

# Note: territories coords change over the year, hence we save the year near the territory names
df_province["TerritorioAnno"] = df_province["Territorio"] + df_province['TIME'].astype(str)
for year in range(2014, 2018):
    map_df[year]["TerritorioAnno"] = map_df[year]["DEN_PCM"] + str(year)


## Change the coordinate system, then create, save and load the GeoJson data

# Function to convert (project) coordinates to latitude/longitude
def convertCrsToLatLong(inputGeopandasDf):
    outputGeopandasDf = inputGeopandasDf.set_geometry("geometry") # The original geometry column is replaced with "geometry" (if it was different).
    outputGeopandasDf = outputGeopandasDf.to_crs("EPSG:4326")
    return outputGeopandasDf

# Union of geodata over years
map_df = pd.concat([convertCrsToLatLong(map_df[year]) for year in range(2014,2018)])

# Create a GeoJson file from the *.shp and read it
geoJsonPath = geoJsonFolder + '.json'
if not os.path.exists(geoJsonFolder):
    os.makedirs(geoJsonFolder)
map_df.to_file(geoJsonPath, driver="GeoJSON")
with open(geoJsonPath, encoding="utf-8") as geofile:
    geoJsonData = json.load(geofile)

print(df_province.head())
# print(geoJsonData)


# Choropleth with slider animation over years
fig = px.choropleth(
    data_frame=df_province, 
    geojson=geoJsonData, 
    locations='TerritorioAnno',                 # name of dataframe column
    hover_name='Territorio',
    hover_data={'Value':True, 'TerritorioAnno':False},
    featureidkey='properties.TerritorioAnno',   # path to field in GeoJSON feature object with which to match the values passed in to locations
    color='Value',
    color_continuous_scale="Magma",
    center={"lat": 42, "lon": 13},
    labels={'Value':'Salary in € ', 'TIME':'Year'},
    projection='mercator',
    animation_frame="TIME"
)
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 100
fig.update_geos(showcountries=False, showcoastlines=False, showland=False, fitbounds="locations")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show("notebook")